In [43]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import os

from IPython.display import display, HTML
BASE_FOLDER = ".."
CONFIGURATION_INPUT_FILE = "configuration-input-new.csv"

In [44]:
def list_directories(directory):
    return [d for d in os.listdir(directory) if os.path.isdir(os.path.join(directory, d))]

In [45]:
def most_recent_directory(directory):
    dirs = list_directories(directory)
    if not dirs:
        return None
    
    most_recent = max(dirs, key=lambda d: os.path.getctime(os.path.join(directory, d)))
    return os.path.join(directory, most_recent)

In [46]:
def mean_of_chunks(series, chunk_size):
    # Explicitly setting numeric_only to True to avoid FutureWarning about deprecation
    return series.groupby(np.arange(len(series)) // chunk_size).mean(numeric_only=True)

In [73]:
def retrieve_metrics():
    input_path = f"{BASE_FOLDER}/input/{CONFIGURATION_INPUT_FILE}"
    with open(input_path, 'r') as file:
        lines = file.readlines()
        second_line = lines[1]  # get the second line
        second_line_elements = second_line.split(',')  # split the line by comma

    df = pd.read_csv(input_path, sep=",")
    all_metrics = []
    read_value = ""
    found_number = False
    index_of_metrics = len(df.columns) - 2

    while not found_number:
        read_value = df.iloc[0, index_of_metrics]
        try:
            int(read_value)
            found_number = True
        except ValueError:
            all_metrics.append(df.iloc[0, index_of_metrics])
            index_of_metrics -= 1

    return all_metrics[::-1]

all_metrics = retrieve_metrics()

In [75]:
all_performance_metrics = ['cpu_limit', 'cpu_usage', 'cpu_demand', 'cpu_utilization', 'cpu_time_active', 'cpu_time_idle', 'cpu_time_steal', 'cpu_time_lost']
all_energy_metrics = ['power_total']
        
selected_performance_metrics = [metric for metric in all_metrics if metric in all_performance_metrics]
selected_energy_metrics = [metric for metric in all_metrics if metric in all_energy_metrics]

print(selected_performance_metrics)
print(selected_energy_metrics)

['cpu_limit', 'cpu_usage', 'cpu_utilization', 'cpu_time_active']
['power_total']


In [16]:
path = f"{BASE_FOLDER}/output/host/workload=bitbrains-small/seed=0/"

recent_dir = most_recent_directory(path)
simulation_files = os.listdir(recent_dir)
simulation_data = []

for file in simulation_files:
    simulation_data.append(
        mean_of_chunks(pd.read_parquet(os.path.join(recent_dir, file)), 1000)
    )
    
simulation_data[0]

,cpu_count,mem_capacity,guests_terminated,guests_running,guests_error,guests_invalid,cpu_limit,cpu_usage,cpu_demand,cpu_utilization,cpu_time_active,cpu_time_idle,cpu_time_steal,cpu_time_lost,power_total,uptime,downtime
0,12.440000,99520.000000,0.0,4.889000,0.0,0.0,38367.280000,9598.129214,9598.129214,0.381752,973.224000,2758.76700,29.721,0.0,77177.762211,300000.000000,0.0
1,12.440000,99520.000000,0.0,4.889000,0.0,0.0,38367.280000,12066.561652,12066.561652,0.488412,1226.278000,2505.72200,50.879,0.0,81974.557057,300000.000000,0.0
2,12.448000,99584.000000,0.0,4.889000,0.0,0.0,38395.040000,13086.369981,13086.369981,0.518187,1325.736000,2408.66400,78.209,0.0,83316.468652,300000.000000,0.0
3,12.440000,99520.000000,0.0,4.889000,0.0,0.0,38367.280000,9127.604039,9127.604039,0.359644,924.555000,2807.44500,0.299,0.0,76187.646193,300000.000000,0.0
4,12.464000,99712.000000,0.0,4.889000,0.0,0.0,38446.240000,3909.118420,3909.118420,0.134652,388.991000,3350.20900,2.259,0.0,66056.400984,300000.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,12.440000,99520.000000,0.0,5.334000,0.0,0.0,38367.280000,2400.631512,2400.631512,0.076533,236.434000,3495.56800,0.000,0.0,63443.570806,300000.000000,0.0
74,12.448000,99584.000000,0.0,5.333000,0.0,0.0,38395.040000,1798.723409,1798.723409,0.046372,172.820000,3561.57700,0.000,0.0,62083.321033,300000.000000,0.0
75,12.440000,99520.000000,0.0,5.333000,0.0,0.0,38367.280000,5735.523457,5735.523457,0.207959,573.358000,3158.64200,0.000,0.0,69346.906178,300000.000000,0.0
76,12.464000,99712.000000,0.0,5.334000,0.0,0.0,38446.240000,4542.487654,4542.487654,0.166209,453.150000,3286.05300,2.823,0.0,67443.474125,300000.000000,0.0
